# Helical Compression Spring

<table><tr>
<td>
<img src="./img/Helical_Compression_Spring/Helical_Compression_Spring.png" width=300"/> </td>
<td>
<img src="./img/Helical_Compression_Spring/Helical_Compression_Spring_end_type.png" width=400"/> </td>
</tr></table>

Importing the relevant Classes

In [1]:
from me_toolbox.springs import HelicalCompressionSpring, Spring

## Geometric properties

d - Wire diamter<br>
D - Spring mean diameter<br>
ends - How the spring's ends look

In [2]:
d = 6   # [mm]
D = 50  # [mm]
ends = 'squared and ground'

## Material properties

Table 10-4 from Shigley:
<table><tr>
<td>
<img src="./img/Helical_Compression_Spring/Tensile_strength_estimation.png" width=360"/> </td>
<td>
<img src="./img/Helical_Extension_Spring/Mechanical_properties_of_some_spring_wires_-_table_10-5.png" width=300"/> </td>
</tr></table>

Material options for use in Spring.material_prop method are:<br>
  * music wire<br>
  * oq&t wire<br>
  * hard-drawn wire<br>
  * chrome-vanadium wire<br>
  * chrome-silicon wire<br>
  * 302 stainless wire<br>
  * phosphore-bronze wire<br>

G - Shear modulus<br>
E - Elastic modulus<br>
Sut - Ultimate tensile strength<br>

In [3]:
# music wire
G = 75e3  # [MPa]
E = 205e3  # [MPa]
Sut = Spring.material_prop('music wire', d, metric=True, verbose=False)
print(f"Sut = {Sut:.2f}")

Sut = 1705.13


Table 10-6 from Shigley:
<table><tr>
<td> <img src="./img/Helical_Compression_Spring/present_of_tensile_strength.png" width=600"/> </td>
</tr></table>


In [4]:
yield_percent = 0.45

## Forces and spring rate

k - Spring rate<br>
Famx - Maximum Force<br>
Fmin - Minimum Force<br>

In [5]:
k = 6  # [N/mm]
Fmax = 500  # [N]
Fmin = 100  # [N]

## Optional parameters

Set removal - a process where the spring is pressed to its solid length - **not used when dynamic loading occurs**<br>
Shot peening - a cold working process in which the surface of the spring is impacted with a small spherical media called shot in order to increase the fatigue resistance.<br>
rho - Material density<br>
zeta - overrun safety factor


In [6]:
removed_set=False
peened=True
rho = 7800  # [kg/m^3]
omega=None
anchors='fixed-hinged'
z=0.25

## Defining the spring

Required parameters:
  * max_force - The maximum force in $[N]$
  * wire_diameter - The wire diameter in $[mm]$
  * spring_diameter - The spring diameter in $[mm]$
  * ultimate_tensile_strength - The ultimate tensile strength in $[MPa]$
  * shear_yield_precent - the precentage of the shear yield strength(Ssy) from the ultimate tensile strength (Sut)
  * shear_modulus - The material shear modulus in $[MPa]$
  * elastic_modulus - The material elastic modulus in $[MPa]$
  * end_type - The shape of the ends of the spring (used to calculate number of coils)
  * spring_rate - The spring rate in $[N/mm]$

Optional parameters: <br>
(the bold parameters are the defaults)

  * set_removed=**False** - If the spring's set has been removed (brought to solid state)
  * shot_peened=**False** - If the spring underwent shot peening
  * density=**None** - The Material density in $[kg/mm^3]$
  * zeta=**0.15** - The overrun safety factor $(F_{solid}=(1+\zeta) \cdot F_{max})$


In [7]:
spring = HelicalCompressionSpring(max_force=Fmax,
                                  wire_diameter=d,
                                  spring_diameter=D,
                                  ultimate_tensile_strength=Sut,
                                  shear_yield_percent=yield_percent,
                                  shear_modulus=G,
                                  elastic_modulus=E,
                                  end_type=ends,
                                  spring_rate=k,
                                  set_removed=removed_set,
                                  shot_peened=peened,
                                  density=rho,
                                  zeta=z)

In [8]:
spring.check_design();

Note: active_coils=16.08 is not in range [3,15],this can cause non linear behavior


### Calculated geometric properties

In [9]:
print(f"Free Length (L0) = {spring.free_length:.2f}\n"
      f"Solid Length (Ls) = {spring.solid_length:.2f}\n"
      f"Active Coils (Na) = {spring.active_coils:.2f}\n"
      f"Total Coils (Nt) = {spring.total_coils:.2f}\n"
      f"End Coils (Ne) = {spring.end_coils:.2f}\n"
      f"Pitch (p) = {spring.pitch:.2f}\n"
      f"Max Diflaction = {spring.max_deflection:.2f}")

Free Length (L0) = 212.67
Solid Length (Ls) = 108.51
Active Coils (Na) = 16.08
Total Coils (Nt) = 18.08
End Coils (Ne) = 2.00
Pitch (p) = 12.48
Max Diflaction = 83.33


In [10]:
print(f"Force at solid length (Fsolid) = {spring.Fsolid:.2f}\n"
      f"Max Shear Stress = {spring.max_shear_stress:.2f}")

Force at solid length (Fsolid) = 625.00
Max Shear Stress = 346.63


### Calculated material properties

In [11]:
print(f"Shear yield strength (Ssy) = {spring.shear_yield_strength:.2f}\n"
      f"Shear ultimate strength (Ssu) = {spring.shear_ultimate_strength:.2f}\n")

Shear yield strength (Ssy) = 767.31
Shear ultimate strength (Ssu) = 1142.44



## Static analysis

In [12]:
print(f"static safety factor = {spring.static_safety_factor():.2f}")
print(f"solid safety factor = {spring.static_safety_factor(solid=True):.2f}")

static safety factor = 2.21
solid safety factor = 1.77


## Fatigue analysis

> Note: The verbose option can be used to print more information

In [13]:
nf, ns, N, Sf= spring.fatigue_analysis(max_force=Fmax, min_force=Fmin, reliability=99.999, verbose=False)
print(f"Fatigue safety factor (nf) = {nf:}\n"
      f"Static safety factor (ns) = {ns:}\n"
      f"Number of cycles untill failure (N) = {N:.2f}\n"
      f"Stress at failure (Sf) = {Sf}")

Fatigue safety factor (nf) = 1.6801675569311425
Static safety factor (ns) = 2.2136490752395956
Number of cycles untill failure (N) = inf
Stress at failure (Sf) = None


#### Increasing the maximum force
After increasing the max force the fatigue safety factor is below 1 so the number of <br>
cycles is not infinite any more and can be calculated as well as the stress at failure<br>

**As shown:**

In [14]:
Fmax = 1000
nf, ns, N, Sf= spring.fatigue_analysis(max_force=Fmax, min_force=Fmin, reliability=99.999, verbose=False)
print(f"Fatigue safety factor (nf) = {nf}\n"
      f"Static safety factor (ns) = {ns}\n"
      f"Number of cycles untill failure (N) = {N}\n"
      f"Stress at failure (Sf) = {Sf}")

Fatigue safety factor (nf) = 0.7915777277160205
Static safety factor (ns) = 1.1068245376197978
Number of cycles untill failure (N) = 97221.28061023176
Stress at failure (Sf) = 468.23730587509914


## Buckling analysis

Buckling test, Table 10-2 from Shigley:
<table><tr>
<td> <img src="./img/Helical_Compression_Spring/Buckling_modes.png" width=400"/> </td>
<td> <img src="./img/Helical_Compression_Spring/Buckling_modes_table.png" width=500"/> </td>
</tr></table>


> Note: The verbose option can be used to print more information

In [15]:
spring.buckling('fixed-hinged', verbose=True);

Buckling is accruing, the max safe length = 190.14,  but the free_length = 212.67


## Frequency analysis

density - Spring's material density in $\frac{kg}{m^3}$<br>
working_frequency - The expacted frequency in wich the spring expexcted to operate in $Hz$

In [16]:
density = rho  # [kg/m^3]
working_frequency = 0.0005  # [Hz]

The natural frequency must be greater than 20 times the working frequency<br>
$(f_{natural} > 20 \cdot f_{working})$
> Note: The verbose option can be used to print more information

In [17]:
spring.natural_frequency(density, working_frequency, verbose=True)

The spring's natural frequency for fixed ends is much grater than the working frequency 
which is good

The spring's natural frequency for one fixed and one free ends is muchgrater than the working frequency 
which is good



{'fixed-fixed': 0.0520715383736995, 'fixed-free': 0.0260357691868497}

## Spring Rate From Geometry

In [18]:
k = spring.calc_spring_rate(wire_diameter = 6 , 
                            spring_diameter = 50, 
                            total_coils = 18, 
                            end_type = 'squared and ground', 
                            shear_modulus = 75e3)
print(f"Spring Rate = {k:.2f}")

Spring Rate = 6.03
